# Data preparation

In [35]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
import time
import bs4
import re
import os 
import urllib.request, urllib.error, urllib.parse
from multiprocessing import Pool
import urllib.request
import json
from bs4 import BeautifulSoup
import mysql.connector
from sqlalchemy import create_engine
import pymysql

In [36]:
# import the dataset with the profile links
profile_urls = pd.read_csv('RSC reviews with profile ids.csv')

In [37]:
profile_urls.columns

Index(['source', 'product', 'PART NUMBER_custom', 'SKU_custom',
       'analysis_purpose_custom_custom', 'flag_custom', 'special_name_custom',
       'test_field2_custom', 'test_field3_custom', 'name', 'date', 'status',
       'sentiment', 'topic', 'notes', 'profile', 'author', 'verified', 'vine',
       'stars', 'pvotes', 'tvotes', 'title', 'text', 'image', 'video',
       'reviewid', 'reviewlink', 'parent', 'statusdt', 'statustime',
       'helpfulcount', 'commenttext', 'commentauthor', 'officialcomment',
       'totalcomments', 'commentts', 'commentdatestring', 'inputtime'],
      dtype='object')

In [38]:
# select related columns
profile_urls = profile_urls[['source','product','profile','author']]

In [39]:
# transfer source name for next step (concat urls)
list_source = list(profile_urls['source'])   #find the source col and transfer to list
for i in list_source:
    if i == 'amazon.fr':
        profile_urls['source'] = profile_urls['source'].replace(i, 'amazon-fr')
for i in list_source:
    if i == 'amazon.ca':
        profile_urls['source'] = profile_urls['source'].replace(i, 'amazon-ca')
for i in list_source:
    if i == 'amazon.de':
        profile_urls['source'] = profile_urls['source'].replace(i, 'amazon-de')
for i in list_source:
    if i == 'amazon.uk':
        profile_urls['source'] = profile_urls['source'].replace(i, 'amazon-uk')

In [40]:
# create profile urls for scraping
profile_urls['product_url'] = 'https://reviewmeta.com/'+ profile_urls['source'] +'/'+ profile_urls['product']

In [41]:
profile_urls.head()

,source,product,profile,author,product_url
0,amazon-ca,B078N8NR7G,https://www.amazon.ca/gp/profile/amzn1.account...,Amazon Customer,https://reviewmeta.com/amazon-ca/B078N8NR7G
1,amazon-ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,nathalie,https://reviewmeta.com/amazon-ca/B01HO8U5NC
2,amazon-ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,Conure Mum,https://reviewmeta.com/amazon-ca/B01HO8U5NC
3,amazon-ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,Wayne Smith,https://reviewmeta.com/amazon-ca/B01HO8U5NC
4,amazon-ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,Rob Self,https://reviewmeta.com/amazon-ca/B01HO8U5NC


In [42]:
# extract unique urls for each product (amazon.com, amazon.ca etc are separated)
urls_by_product = profile_urls.groupby(["product_url", "source", "product"])['profile'].agg('nunique')
urls_by_product = urls_by_product.reset_index()

In [43]:
# sort the product with most profile info (maybe can change to other criteria)
urls_by_product = urls_by_product.sort_values(['profile', 'product'], ascending=False).reset_index(drop = True)

In [44]:
urls_by_product.head()

,product_url,source,product,profile
0,https://reviewmeta.com/amazon/B0721735K9,amazon,B0721735K9,1461
1,https://reviewmeta.com/amazon/B00VIXRB6O,amazon,B00VIXRB6O,1442
2,https://reviewmeta.com/amazon/B00063446M,amazon,B00063446M,1285
3,https://reviewmeta.com/amazon/B00074L4W2,amazon,B00074L4W2,1112
4,https://reviewmeta.com/amazon/B0009811TW,amazon,B0009811TW,847


In [45]:
urls_for_test = urls_by_product.head(10)

In [58]:
urls_for_test.to_csv("urls_by_product.csv", index = False)

# web-scraping

In [139]:
## First do some preparation:
# set user_agent
user_agent = {'User-agent': 'Mozilla/5.0'} 

# get the current directory and creat folders with name "reviewmeta_pages"
folder = os.getcwd() + '\\reviewmeta_pages\\'
if not os.path.exists(folder):
    os.makedirs(folder)

# read the file we just stored
urls_by_product = pd.read_csv('urls_by_product.csv', sep=',')
urls_by_product.head()

,product_url,source,product,profile
0,https://reviewmeta.com/amazon/B0721735K9,amazon,B0721735K9,1461
1,https://reviewmeta.com/amazon/B00VIXRB6O,amazon,B00VIXRB6O,1442
2,https://reviewmeta.com/amazon/B00063446M,amazon,B00063446M,1285
3,https://reviewmeta.com/amazon/B00074L4W2,amazon,B00074L4W2,1112
4,https://reviewmeta.com/amazon/B0009811TW,amazon,B0009811TW,847


In [140]:
#download each page into the folder and name them with "ID+Source" 

for i in range(0,10):
    time.sleep(1) #Hold 1 seconds before the next scrape.
    num=str(i)
    url = urls_by_product.loc[i,'product_url']
    product = urls_by_product.loc[i,'product']
    source = urls_by_product.loc[i,'source']
    path = os.getcwd()
    path = path + '\\reviewmeta_pages\\'
    filename = product +"_"+ source + '.htm'
    response = requests.get(url, headers = user_agent)
    with open(path + filename, 'w', encoding="utf8") as file:
        file.write(response.text)
    time.sleep(2)

print('Completed')

Completed


# extract attributes --> Need improvement

In [238]:
product = urls_by_product['product']
source = urls_by_product['source']

soup = {}
for i in range(0,10):
    num=str(i)
    filename = product[i] +"_"+ source[i] + '.htm'
    text=open(path + filename,'r', encoding = 'utf8').read() #Read the text from the file.
    soup[i]=bs4.BeautifulSoup(text, 'html.parser') #Create a beautifulsoup object using the txt we got.

In [239]:
test = urls_by_product # for test

In [245]:
# extracting info from pages,and generate product-based table
for i in range(0,10):
    try:
        # sample reviews - bad
        tag[i] = soup[i].find_all('span', class_='more-examples-bad')[0]       
        test.loc[i,'bad_1_trust'] = soup[i].find('div', id='bad-reviews').find('div').find_all('b')[1].get_text()     
        test.loc[i,'bad_1_link'] = soup[i].find('div', id='bad-reviews').find_all('a')[0].get('href')
        
        # contents when unfold the 'show more' botton
        # more reviews - bad
        test.loc[i,'bad_2_trust'] = tag[i].find_all('div', {'id':'sample_reviews'})[0].find_all('b')[1].get_text()
        test.loc[i,'bad_2_link'] = tag[i].find_all('div', {'id':'sample_reviews'})[0].find_all('a')[0].get('href')
        
        test.loc[i,'bad_3_trust'] = tag[i].find_all('div', {'id':'sample_reviews'})[1].find_all('b')[1].get_text()
        test.loc[i,'bad_3_link'] = tag[i].find_all('div', {'id':'sample_reviews'})[1].find_all('a')[0].get('href')
        
        test.loc[i,'bad_4_trust'] = tag[i].find_all('div', {'id':'sample_reviews'})[2].find_all('b')[1].get_text()    
        test.loc[i,'bad_4_link'] = tag[i].find_all('div', {'id':'sample_reviews'})[2].find_all('a')[0].get('href')
        
        test.loc[i,'bad_5_trust'] = tag[i].find_all('div', {'id':'sample_reviews'})[3].find_all('b')[1].get_text()
        test.loc[i,'bad_5_link'] = tag[i].find_all('div', {'id':'sample_reviews'})[3].find_all('a')[0].get('href')
        
        test.loc[i,'bad_6_trust'] = tag[i].find_all('div', {'id':'sample_reviews'})[4].find_all('b')[1].get_text()
        test.loc[i,'bad_6_link'] = tag[i].find_all('div', {'id':'sample_reviews'})[4].find_all('a')[0].get('href')
                
        test.loc[i,'bad_7_trust'] = tag[i].find_all('div', {'id':'sample_reviews'})[5].find_all('b')[1].get_text()
        test.loc[i,'bad_7_link'] = tag[i].find_all('div', {'id':'sample_reviews'})[5].find_all('a')[0].get('href')
        
        test.loc[i,'bad_8_trust'] = tag[i].find_all('div', {'id':'sample_reviews'})[6].find_all('b')[1].get_text()
        test.loc[i,'bad_8_link'] = tag[i].find_all('div', {'id':'sample_reviews'})[6].find_all('a')[0].get('href')
        
        test.loc[i,'bad_9_trust'] = tag[i].find_all('div', {'id':'sample_reviews'})[7].find_all('b')[1].get_text()    
        test.loc[i,'bad_9_link'] = tag[i].find_all('div', {'id':'sample_reviews'})[7].find_all('a')[0].get('href')
        
        test.loc[i,'bad_10_trust'] = tag[i].find_all('div', {'id':'sample_reviews'})[8].find_all('b')[1].get_text()
        test.loc[i,'bad_10_link'] = tag[i].find_all('div', {'id':'sample_reviews'})[8].find_all('a')[0].get('href') 
    except:
        print(i)
test.head()

,product_url,source,product,profile,bad_1_trust,bad_2_trust,bad_3_trust,bad_1_link,bad_2_link,bad_3_link,...,bad_6_trust,bad_6_link,bad_7_trust,bad_7_link,bad_8_trust,bad_8_link,bad_9_trust,bad_9_link,bad_10_trust,bad_10_link
0,https://reviewmeta.com/amazon/B0721735K9,amazon,B0721735K9,1461,0,0,0,https://smile.amazon.com/gp/customer-reviews/R...,https://smile.amazon.com/gp/customer-reviews/R...,https://smile.amazon.com/gp/customer-reviews/R...,...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...
1,https://reviewmeta.com/amazon/B00VIXRB6O,amazon,B00VIXRB6O,1442,0,0,32,https://smile.amazon.com/gp/customer-reviews/R...,https://smile.amazon.com/gp/customer-reviews/R...,https://smile.amazon.com/gp/customer-reviews/R...,...,32,https://smile.amazon.com/gp/customer-reviews/R...,32,https://smile.amazon.com/gp/customer-reviews/R...,32,https://smile.amazon.com/gp/customer-reviews/R...,32,https://smile.amazon.com/gp/customer-reviews/R...,32,https://smile.amazon.com/gp/customer-reviews/R...
2,https://reviewmeta.com/amazon/B00063446M,amazon,B00063446M,1285,0,0,0,https://smile.amazon.com/gp/customer-reviews/R...,https://smile.amazon.com/gp/customer-reviews/R...,https://smile.amazon.com/gp/customer-reviews/R...,...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...
3,https://reviewmeta.com/amazon/B00074L4W2,amazon,B00074L4W2,1112,0,0,0,https://smile.amazon.com/gp/customer-reviews/R...,https://smile.amazon.com/gp/customer-reviews/R...,https://smile.amazon.com/gp/customer-reviews/R...,...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...
4,https://reviewmeta.com/amazon/B0009811TW,amazon,B0009811TW,847,0,0,0,https://smile.amazon.com/gp/customer-reviews/R...,https://smile.amazon.com/gp/customer-reviews/R...,https://smile.amazon.com/gp/customer-reviews/R...,...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...,0,https://smile.amazon.com/gp/customer-reviews/R...


## from here on: below are not finished yet

In [248]:
# write file to local folder
test.to_csv("test.csv", index = False)

In [ ]:
# extracting info from pages,and generate product-based
# similar process for good reviews, but bugs appear, will fix it later
for i in range(0,10):
    try:
        g_tag[i] = soup[i].find_all('span', class_='more-examples-good')[0]
        test.loc[i,'good_1_trust'] = soup[i].find('div', id='good-reviews').find('div').find_all('b')[1].get_text()     
        test.loc[i,'good_1_link'] = soup[i].find('div', id='good-reviews').find_all('a')[0].get('href')
        
        # more reviews - good
        test.loc[i,'good_2_trust'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[0].find_all('b')[1].get_text()
        test.loc[i,'good_2_link'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[0].find_all('a')[0].get('href')
        
        test.loc[i,'good_3_trust'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[1].find_all('b')[1].get_text()
        test.loc[i,'good_3_link'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[1].find_all('a')[0].get('href')
        
        test.loc[i,'good_4_trust'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[2].find_all('b')[1].get_text()    
        test.loc[i,'good_4_link'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[2].find_all('a')[0].get('href')
        
        test.loc[i,'good_5_trust'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[3].find_all('b')[1].get_text()
        test.loc[i,'good_5_link'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[3].find_all('a')[0].get('href')
        
        test.loc[i,'good_6_trust'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[4].find_all('b')[1].get_text()
        test.loc[i,'good_6_link'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[4].find_all('a')[0].get('href')
                
        test.loc[i,'good_7_trust'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[5].find_all('b')[1].get_text()
        test.loc[i,'good_7_link'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[5].find_all('a')[0].get('href')
        
        test.loc[i,'good_8_trust'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[6].find_all('b')[1].get_text()
        test.loc[i,'good_8_link'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[6].find_all('a')[0].get('href')
        
        test.loc[i,'good_9_trust'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[7].find_all('b')[1].get_text()    
        test.loc[i,'good_9_link'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[7].find_all('a')[0].get('href')
        
        test.loc[i,'good_10_trust'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[8].find_all('b')[1].get_text()
        test.loc[i,'good_10_link'] = g_tag[i].find_all('div', {'id':'sample_reviews'})[8].find_all('a')[0].get('href')

    except:
        print(i)
test